In [3]:
from pathlib import Path
import json
import time
from datetime import datetime

import numpy as np
import whisper
from rich import print as rprint
import midii

import preprocess_svs as ps
from preprocess_svs import gv
from preprocess_svs import LyricNormalizer, SVS_Preprocessor

# GV File Correction

In [4]:
gv_path = "D:/dataset/177.다음색 가이드보컬 데이터"
gv_json_sample = "sample/gv/json"
gv_mid_sample = "sample/gv/midi"
gv_sample_preprocessed = "sample/gv/json_preprocessed"
gv_json_time_adjusted = "D:/dataset/다음색 가이드보컬 데이터 time_adjusted"
gv_json_preprocessed = "D:/dataset/다음색 가이드보컬 데이터 json preprocessed"
midi_filepath = "sample/gv/midi/SINGER_16_10TO29_CLEAR_FEMALE_BALLAD_C0632.mid"
time_adjusted_json_filepath = "sample/gv/json_time_adjusted/SINGER_16_10TO29_CLEAR_FEMALE_BALLAD_C0632.json"
filled_time_gaps_json_filepath = "sample/gv/json_filled_time_gaps/SINGER_16_10TO29_CLEAR_FEMALE_BALLAD_C0632.json"

In [5]:
print(len(list(ps.get_files(gv_path, "mid"))))

0


In [6]:
mid = midii.MidiFile(midi_filepath, convert_1_to_0=True)
tempo_rank = mid.tempo_rank()
print(tempo_rank)
print(ps.calculate_top_tempo_percentage(tempo_rank))

[(857142, 790)]
100.0


## Analysis Tempo Deviation

- json 을 처리하려면 quantize 를 위한 tempo 가 필요한데 json 에는 tempo 정보가 없음 
- -> tempo rank 검사 
- -> tempo 가 변하지 않는다는 충분한 보장
- -> dominate tempo 를 채택하여 quantize 해도 된다

In [7]:
# ps.tempo_statistics(gv_path, parallel=True, verbose=True)

- -> 이전 end_time 이 현재 start_time 보다 큰 경우가 있음 
- -> 이전 end_time 에 현재 start_time 을 맞추면, 뒤따라오는 메시지들의 sync 가 다 틀어짐 
- -> 이전 end_time 을 현재 start_time 에 맞춰주는 게 더 나음

## Verify notes sorted by time 

In [8]:
# gv.verify_json_notes_sorted_by_time(gv_path, parallel=True)

In [9]:
def adjust_note_times_sample():
    gv_path = "sample/gv/json"
    for json_path in ps.get_files(gv_path, "json"):
        p_orig = Path(json_path)
        out_path = p_orig.parent.parent / "json_time_adjusted" / p_orig.name
        out_path.parent.mkdir(exist_ok=True, parents=True)
        print(f"adjust time of \n{json_path}")
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        notes = data.get("notes")
        processed_notes = gv.adjust_note_times(notes)
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(processed_notes, f, ensure_ascii=False, indent=4)
        print(f"saved to \n{out_path}")

In [10]:
adjust_note_times_sample()

adjust time of 
sample/gv/json/SINGER_66_30TO49_HUSKY_MALE_DANCE_C2835.json
saved to 
sample/gv/json_time_adjusted/SINGER_66_30TO49_HUSKY_MALE_DANCE_C2835.json
adjust time of 
sample/gv/json/SINGER_16_10TO29_CLEAR_FEMALE_BALLAD_C0632.json
saved to 
sample/gv/json_time_adjusted/SINGER_16_10TO29_CLEAR_FEMALE_BALLAD_C0632.json


## fill silence note between notes

In [11]:
print(time_adjusted_json_filepath)
print(filled_time_gaps_json_filepath)

sample/gv/json_time_adjusted/SINGER_16_10TO29_CLEAR_FEMALE_BALLAD_C0632.json
sample/gv/json_filled_time_gaps/SINGER_16_10TO29_CLEAR_FEMALE_BALLAD_C0632.json


In [12]:
gv.fill_time_gaps_save(time_adjusted_json_filepath, filled_time_gaps_json_filepath)

save:
sample/gv/json_filled_time_gaps/SINGER_16_10TO29_CLEAR_FEMALE_BALLAD_C0632.json


## verify correspondence json vs wav vs mid 

In [13]:
jsons = ps.get_files(gv_path, "json", sort=True)
mids = ps.get_files(gv_path, "mid", sort=True)
wavs = ps.get_files(gv_path, "wav", sort=True)
rprint(gv.verify_files_coherent(jsons, mids))
rprint(gv.verify_files_coherent(wavs, mids))
rprint(gv.verify_files_coherent(jsons, wavs))

([], [])

([], [])

([], [])

## Remove abnormal files

In [14]:
gv.remove_abnormal_file(gv_path)

([], [])

# GV Preprocessing

In [15]:
json_filepath = "sample/gv/json_preprocessed/SINGER_16_10TO29_CLEAR_FEMALE_BALLAD_C0632.json"
split_json_filepath = "sample/gv/split_json/SINGER_16_10TO29_CLEAR_FEMALE_BALLAD_C0632.json"
preprocessed_gv_path = "preprocessed_gv/"
preprocessed_gv_duration_path = "preprocessed_gv/duration"
preprocessed_gv_pitch_path = "preprocessed_gv/pitch"
preprocessed_gv_wav_path = "preprocessed_gv/wav"

## Step 1 - preprocess gv json

- gv json -> adjust note times + fill time gaps + quantization + frames
- embed coherent json format(sharing with mssv)

In [16]:
print(gv_json_sample, gv_mid_sample, gv_sample_preprocessed)

sample/gv/json sample/gv/midi sample/gv/json_preprocessed


In [17]:
gv.preprocess_json(
    gv_json_sample, gv_mid_sample, gv_sample_preprocessed, parallel=True
)

In [18]:
# gv.preprocess_json(
#     gv_path,
#     gv_path,
#     gv_json_preprocessed,
#     parallel=True,
# )

## Step 2 - split notes by silence

In [19]:
# split_json = ps.split_json_by_silence(json_filepath, min_length=6)
# split_json_filepath = Path(split_json_filepath)
# split_json_filepath.parent.mkdir(exist_ok=True, parents=True)
# with open(split_json_filepath, "w", encoding="utf-8") as f:
#     json.dump(split_json, f, indent=4, ensure_ascii=False)

In [20]:
gv.split_json(json_filepath, split_json_filepath)

In [21]:
json_dirpath = 'sample/gv/json_preprocessed'
split_json_dirpath = 'sample/gv/split_json'
gv.split_jsons(json_dirpath, split_json_dirpath)

## Step 3 - save duration, pitch as npy file, split audio, save metadata

In [22]:
wav_filepath = "sample/gv/wav/SINGER_16_10TO29_CLEAR_FEMALE_BALLAD_C0632.wav"

In [23]:
metadata_list = []
metadata_list.append(
    gv.preprocess_one(
        wav_filepath,
        split_json_filepath,
        preprocessed_gv_path
    )
)
preprocessed_gv_path = Path(preprocessed_gv_path)
preprocessed_gv_path.mkdir(exist_ok=True, parents=True)
with open(f"{preprocessed_gv_path}/metadata.txt", "w", encoding="utf-8") as f:
    f.write("".join(metadata_list))

In [24]:
wav_dirpath = 'sample/gv/wav/'
split_json_dirpath = 'sample/gv/split_json/'
gv.save_duration_pitch_metadata_split_audio(
    wav_dirpath,
    split_json_dirpath,
    preprocessed_gv_path
)

## Normalizer 사용 설명

### 1. lyric_normalizer.py의 LyricNormalizer 클래스 import
### 2. LyricNormalizer 객체 생성
### 3. LyricNormalizer.normalize_lyrics() 함수 사용
#### &emsp; Input: GT(whisper result), 원본 가사, pitch sequence, duration sequence
#### &emsp; Output: 정규화 가사, pitch sequence, duration sequence, 정규화 정보를 담은 dictionary


In [25]:
preprocessor = SVS_Preprocessor(
    base_path="preprocessed_gv",
    model_name="large-v3",
    device="cpu",
    language="ko",
)

In [26]:
preprocessor.process_all_files()

/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:124: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:124: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Error in DTW: gt_text: 처음엔너일거라생각못했어
raw_text: 모오스읍처어으매앤너어이일거라아새앵각모오테써어
DTW 행렬의 위치 [0, 13]에서 잘못된 경로 정보를 발견했습니다. 2개의 정수로 구성된 튜플이 필요하지만, 다음을 받았습니다: None


/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:124: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:124: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/sit

Error in DTW: gt_text: 내가아는그녀와있을네모습
raw_text: 내애가아아니인그으녀어와이쓰을니이모오스읍
DTW 행렬의 위치 [0, 13]에서 잘못된 경로 정보를 발견했습니다. 2개의 정수로 구성된 튜플이 필요하지만, 다음을 받았습니다: None


/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:124: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:124: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/sit

Error in DTW: gt_text: 우린다갈수있나왜또너는내게왜이리달콤해
raw_text: 우우리인다아가알쑤우이있나아왜애또오너어는내애게에왜이리이달코옴해애왜애
DTW 행렬의 위치 [0, 32]에서 잘못된 경로 정보를 발견했습니다. 2개의 정수로 구성된 튜플이 필요하지만, 다음을 받았습니다: None


/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:124: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:124: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/home/ccsss/repo/preprocess-svs-dataset/.venv/lib/python3.12/sit


Writing normalized metadata to: preprocessed_gv/metadata.txt
Done.

Starting dataset verification...

=== Starting Dataset Consistency Verification ===

=== Verification Results ===

No errors found!

No warnings!

Dataset verification completed successfully!


In [30]:
preprocessor.verify_dataset_consistency()


=== Starting Dataset Consistency Verification ===

=== Verification Results ===

No errors found!

No warnings!


{'errors': [], 'warnings': []}

# Apply G2pk 

In [31]:
file_path = 'preprocessed_gv/metadata.txt'
ps.g2p_metadata(file_path)

mecab installed
mecab installed
mecab installed
mecab installed
mecab installed
mecab installed
mecab installed
mecab installed
mecab installed
mecab installed
mecab installed
mecab installed
